Data from FE statistics

In [43]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
import sqlite3

In [97]:
# I was going to scrape "https://motorsportstats.com/series/abb-fia-formula-e-world-champioonship/summary/", but there doesnt seem to be 
# a way to break down specific points finishes so I will import a pre-existing .csv that contains the same data. 

drivers = pd.read_csv("drivers.csv", encoding='ISO-8859-1') # very annoying error while imporing had to be fixed for drivers and eprix
eprix = pd.read_csv("eprix.csv", encoding='ISO-8859-1')
powertrains = pd.read_csv("powertrains.csv")
races = pd.read_csv("races.csv")
results = pd.read_csv("results.csv")
seasons = pd.read_csv("seasons.csv")
teams = pd.read_csv("teams.csv")

In [111]:
# quicly explore the different files we have

print(drivers.isnull().sum())
print(eprix.isnull().sum())
print(powertrains.isnull().sum())
print(races.isnull().sum())
print(results.isnull().sum())
print(seasons.isnull().sum())
print(teams.isnull().sum())

# Look like there are some empty items in "results" but that is to be expected

driverId       0
firstname      0
lastname       0
nationality    0
dtype: int64
eprixId          0
eprixRef         0
circuit          0
location         0
country          0
racesPerTrack    0
dtype: int64
powertrainsId    0
powertrain       0
team             0
season           0
dtype: int64
Round       0
ePrix       0
Country     0
Track       0
Date        0
raceId      0
seasonId    0
dtype: int64
resultsId      0
seasonId       0
raceId         0
pos            0
number         0
teamId         0
driverId       0
laps          48
time          47
gap          586
interval     578
kp/h          70
best          67
lap          115
dtype: int64
season       0
yearstart    0
yearend      0
url          1
dtype: int64
teamId         0
teamRef        0
name           0
nationality    0
dtype: int64


In [ ]:
#the url column in "seasons" seems useless so let's remove that
seasons = seasons.drop('url', axis=1)

In [131]:
seasons

,season,yearstart,yearend
0,1,2014,2015
1,2,2015,2016
2,3,2016,2017
3,4,2017,2018
4,5,2018,2019
5,6,2019,2020
6,7,2020,2021
7,8,2021,2022
8,9,2022,2023
9,10,2023,2024


In [147]:
results

,resultsId,seasonId,raceId,pos,number,teamId,driverId,laps,time,gap,interval,kp/h,best,lap
0,1,1,1,DNF,55,3,64,21.0,50:52.3,NaN,NaN,85.525,01:45.1,21.0
1,2,1,1,DNF,9,44,10,14.0,37:24.7,NaN,NaN,77.53,01:45.9,13.0
2,3,1,1,DNF,10,46,74,2.0,41:22.8,NaN,NaN,10.013,01:52.6,2.0
3,4,1,1,DNF,21,26,65,NaN,01:41.3,NaN,NaN,NaN,01:49.5,NaN
4,5,1,1,1,11,6,33,25.0,52:23.4,NaN,NaN,98.863,01:46.7,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,2552,10,119,18,3,19,66,36.0,44:18.4,26.564,0.925,122.436,01:11.5,24.0
2552,2553,10,119,DNF,33,19,71,32.0,41:33.9,NaN,NaN,116.106,01:10.8,28.0
2553,2554,10,119,DNF,18,27,18,25.0,30:55.5,NaN,NaN,122.172,01:11.1,17.0
2554,2555,10,119,DNF,8,31,7,22.0,27:34.3,NaN,NaN,120.739,01:10.7,20.0


In [149]:
races #seems to be a discrepancy between the raceid in the "race" and "results" df

,Round,ePrix,Country,Track,Date,raceId,seasonId
0,1,Beijing ePrix,China,Beijing Olympic Green Circuit,41895,1900-01-01,1
1,2,Putrajaya ePrix,Malaysia,Putrajaya Street Circuit,41965,1900-01-02,1
2,3,Punta del Este ePrix,Uruguay,Punta del Este Street Circuit,41986,1900-01-03,1
3,4,Buenos Aires ePrix,Argentina,Puerto Madero Street Circuit,42014,1900-01-04,1
4,5,Miami ePrix,United States,Biscayne Bay Street Circuit,42077,1900-01-05,1
...,...,...,...,...,...,...,...
128,13,Shanghai E-Prix,China,Shanghai International Circuit,45438,1900-05-08,10
129,14,Portland E-Prix,United States,Portland International Raceway,45472,1900-05-09,10
130,15,Portland E-Prix,United States,Portland International Raceway,45473,1900-05-10,10
131,16,London E-Prix,United Kingdom,ExCeL London,45493,1900-05-11,10


In [145]:
# Create a SQL database to combine these dataframes for later

conn = sqlite3.connect('FE.sqlite') 

drivers.to_sql('drivers',con=conn,index=False,if_exists='replace')
eprix.to_sql('eprix',con=conn,index=False,if_exists='replace')
powertrains.to_sql('powertrains',con=conn,index=False,if_exists='replace')
races.to_sql('races',con=conn,index=False,if_exists='replace')
results.to_sql('results',con=conn,index=False,if_exists='replace')
seasons.to_sql('seasons',con=conn,index=False,if_exists='replace')
teams.to_sql('teams',con=conn,index=False,if_exists='replace')

pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'",con=conn)


,type,name,tbl_name,rootpage,sql
0,table,drivers,drivers,2,"CREATE TABLE ""drivers"" (\n""driverId"" INTEGER,\..."
1,table,eprix,eprix,3,"CREATE TABLE ""eprix"" (\n""eprixId"" INTEGER,\n ..."
2,table,powertrains,powertrains,4,"CREATE TABLE ""powertrains"" (\n""powertrainsId"" ..."
3,table,races,races,5,"CREATE TABLE ""races"" (\n""Round"" INTEGER,\n ""e..."
4,table,results,results,9,"CREATE TABLE ""results"" (\n""resultsId"" INTEGER,..."
5,table,seasons,seasons,48,"CREATE TABLE ""seasons"" (\n""season"" INTEGER,\n ..."
6,table,teams,teams,49,"CREATE TABLE ""teams"" (\n""teamId"" INTEGER,\n ""..."


In [ ]:
# Start to investigate. Who are the winners of each season?




In [ ]:
# Who is the most sucessful driver by wins?

In [ ]:
# Most sucessul team by wins?